In [15]:
#https://ithelp.ithome.com.tw/articles/10240314
#https://github.com/zhixuhao/unet?tab=readme-ov-file
import os
from PIL import Image,ImageOps
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

x_list = []
y_list = []

dir = "./LSCData/A1/"
for file in os.listdir(dir):
    if file.endswith("label.png"):
        y_list.append(np.array(Image.open(dir + file).convert('RGB')))
    else:
        x_list.append(np.array(Image.open(dir + file).convert('RGB')))

# print(x_list)
# print(y_list)
print(len(x_list))
print(len(y_list))



128
128


In [16]:
import matplotlib.pyplot as plt

# plt.imshow(x_list[0])
# plt.axis('off')
# plt.show()

# plt.imshow(y_list[0])
# plt.axis('off')
# plt.show()

In [17]:
train_x = np.array(x_list[:115])
train_y = np.array(y_list[:115])
val_x = np.array(x_list[115:])
val_y = np.array(y_list[115:])

# print(len(train_x))
# print(len(train_y))
# print(len(val_x))
# print(len(val_y))

print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)

(115, 530, 500, 3)
(115, 530, 500, 3)
(13, 530, 500, 3)
(13, 530, 500, 3)


In [18]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import layers

img_size = (530,500,3)
num_classes = 3

def get_model(img_size, num_classes):

    inputs = keras.Input(shape=img_size)

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    conv1 = layers.Conv2D(64, 3, padding="same" ,activation='relu')(inputs)
    conv1 = layers.Conv2D(64, 3, padding="same" ,activation='relu')(conv1)#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!11
    pool1 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv1)

    conv2 = layers.Conv2D(128, 3, padding="same" ,activation='relu')(pool1)
    conv2 = layers.Conv2D(128, 3, padding="same" ,activation='relu')(conv2)#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!11
    pool2 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv2)

    conv3 = layers.Conv2D(256, 3, padding="same" ,activation='relu')(pool2)
    conv3 = layers.Conv2D(256, 3, padding="same" ,activation='relu')(conv3)#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!11
    pool3 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv3)

    conv4 = layers.Conv2D(512, 3, padding="same" ,activation='relu')(pool3)
    conv4 = layers.Conv2D(512, 3, padding="same" ,activation='relu')(conv4)
    conv4_drop = Dropout(0.5)(conv4) #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    pool4 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv4_drop)
    conv5 = layers.Conv2D(1024, 3, padding="same" ,activation='relu')(pool4)
    conv5 = layers.Conv2D(1024, 3, padding="same" ,activation='relu')(conv5)
    conv5_drop = Dropout(0.5)(conv5)

    up1 = UpSampling2D(size = (2,2))(conv5_drop)
    up1 = layers.Conv2D(512, 2, padding="same" ,activation='relu')(up1)
    merge1 = concatenate([conv4_drop,up1], axis = 3)
    conv6 = layers.Conv2D(512, 3, padding="same" ,activation='relu')(merge1)
    conv6 = layers.Conv2D(512, 3, padding="same" ,activation='relu')(conv6)

    up2 = UpSampling2D(size = (2,2))(conv6)
    up2 = layers.Conv2D(256, 2, padding="same" ,activation='relu')(up2)
    up2 = layers.ZeroPadding2D(padding=((0, 0), (0, 1)))(up2) #?????????? padding 
    merge2 = concatenate([conv3,up2], axis = 3)
    conv7 = layers.Conv2D(256, 3, padding="same" ,activation='relu')(merge2)
    conv7 = layers.Conv2D(256, 3, padding="same" ,activation='relu')(conv7)

    up3 = UpSampling2D(size = (2,2))(conv7)
    up3 = layers.Conv2D(128, 2, padding="same" ,activation='relu')(up3)
    up3 = layers.ZeroPadding2D(padding=((1, 0), (0, 0)))(up3) #?????????? padding 
    merge3 = concatenate([conv2,up3], axis = 3)
    conv8 = layers.Conv2D(128, 3, padding="same" ,activation='relu')(merge3)
    conv8 = layers.Conv2D(128, 3, padding="same" ,activation='relu')(conv8)

    up4 = UpSampling2D(size = (2,2))(conv8)
    up4 = layers.Conv2D(64, 2, padding="same" ,activation='relu')(up4)
    merge4 = concatenate([conv1,up4], axis = 3)
    conv9 = layers.Conv2D(64, 3, padding="same" ,activation='relu')(merge4)
    conv9 = layers.Conv2D(64, 3, padding="same" ,activation='relu')(conv9)
    conv9 = layers.Conv2D(2, 3, padding="same" ,activation='relu')(conv9)
    conv10 = layers.Conv2D(num_classes, 1, activation='sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(), loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.summary()

    return model
    
model = get_model(img_size, num_classes)

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 530, 500,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_72 (Conv2D)  │ (None, 530, 500,  │      1,792 │ input_layer_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_73 (Conv2D)  │ (None, 530, 500,  │     36,928 │ conv2d_72[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 265, 250,  │          0 │ conv2d_73[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_74 (Conv2D)  │ (None, 265, 250,  │     73,856 │ max_pooling2d_12… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_75 (Conv2D)  │ (None, 265, 250,  │    147,584 │ conv2d_74[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_13    │ (None, 132, 125,  │          0 │ conv2d_75[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_76 (Conv2D)  │ (None, 132, 125,  │    295,168 │ max_pooling2d_13… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_77 (Conv2D)  │ (None, 132, 125,  │    590,080 │ conv2d_76[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_14    │ (None, 66, 62,    │          0 │ conv2d_77[0][0]   │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_78 (Conv2D)  │ (None, 66, 62,    │  1,180,160 │ max_pooling2d_14… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_79 (Conv2D)  │ (None, 66, 62,    │  2,359,808 │ conv2d_78[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 66, 62,    │          0 │ conv2d_79[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_15    │ (None, 33, 31,    │          0 │ dropout_6[0][0]   │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_80 (Conv2D)  │ (None, 33, 31,    │  4,719,616 │ max_pooling2d_15… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_81 (Conv2D)  │ (None, 33, 31,    │  9,438,208 │ conv2d_80[0][0]   │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 33, 31,    │          0 │ conv2d_81[0][0] 

 Total params: 31,032,843 (118.38 MB)

 Trainable params: 31,032,843 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")

# 設定檢查點 callbacks，模型存檔
callbacks = [
    keras.callbacks.ModelCheckpoint("segmentation.keras", save_best_only=True)
]

# 訓練 15 週期(epoch)
epochs = 15
model.fit(train_x, train_y, verbose=1, epochs=epochs, callbacks=callbacks)

Epoch 1/15


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 530, 500, 3), output.shape=(None, 530, 500, 3)